In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ipywidgets as widgets
from IPython.display import display, HTML
from ipywidgets import HBox, VBox


data = pd.read_csv(r'Dataset\stocks.csv')
# Convert the 'Date' column to datetime format
data['Date'] = pd.to_datetime(data['Date'])


def print_centered(text, font_size):
    centered_text = f"<center><h3 style='font-size: {font_size}px;'>{text}</h3></center>"
    display(HTML(centered_text))

# Example usage
print_centered("Stock Market Analysis", 30)
print()
print()

def print_centered(text, font_size):
    centered_text = f"<div style='text-align: center; font-size: {font_size}px;'>{text}</div>"
    display(HTML(centered_text))


def plot_closing_prices(company_name):
    # Filter the data for the specified company
    company_data = data[data['Symbol'] == company_name]

    # Extract the 'Date' and 'Close' columns for the specified company
    dates = company_data['Date']
    closing_prices = company_data['Close']

    # Plot the closing prices for the specified company
    plt.figure()
    plt.plot(dates, closing_prices)
    plt.xlabel("Date")
    plt.ylabel("Closing Price")
    plt.title(f"{company_name} Stock Closing Prices")
    plt.xticks(rotation=45)
    plt.show()

def calculate_returns_and_plot(share_company, time_period):
    if share_company == '' or time_period == '':
        return

    # Filter the data for the specified company
    share_data = data[data['Symbol'] == share_company]

    # Filter the data based on the selected time period
    if time_period == '1d':
        filtered_data = share_data.tail(1)
    elif time_period == '5d':
        filtered_data = share_data.tail(5)
    elif time_period == '1month':
        filtered_data = share_data.tail(30)
    elif time_period == '1year':
        filtered_data = share_data.tail(365)
    elif time_period == '5years':
        filtered_data = share_data.tail(1825)
    else:
        filtered_data = None

    if filtered_data is not None:
        # Calculate the returns as percentage
        returns = ((filtered_data['Close'].iloc[-1] - filtered_data['Close'].iloc[0]) / filtered_data['Close'].iloc[0]) * 100

        # Display the returns value as percentage
        print(f'Returns for {share_company} ({time_period}): {returns:.2f}%')

        # Calculate the returns for the specific share
        share_df = share_data.copy()
        share_df['Return'] = share_df['Close'].pct_change()

        # Filter the DataFrame based on the selected time duration
        def get_returns_for_duration(duration):
            if duration == '1d':
                return share_df['Return'].iloc[-1] * 100
            elif duration == '5d':
                return share_df['Return'].iloc[-5:].sum() * 100
            elif duration == '1month':
                return share_df['Return'].iloc[-30:].sum() * 100
            elif duration == '1year':
                return share_df['Return'].iloc[-365:].sum() * 100
            elif duration == '5years':
                return share_df['Return'].iloc[-1825:].sum() * 100
            else:
                return None  # Return None for invalid duration

        returns = get_returns_for_duration(time_period)

        if returns is not None:
            with output:
                output.clear_output()
                print()
                print()
                centered_returns = f'Returns for {share_company} ({time_period}): {returns:.2f}%'
                print_centered(centered_returns, 16)
                print()
                print()

        # Plot returns over one year with months on the x-axis

# Plot returns over one year with years on the x-axis

    monthly_returns = share_df.resample('M', on='Date')['Return'].sum() * 100
    years = monthly_returns.index.year.unique()
    plt.figure(figsize=(5, 5))
    plt.plot(years, monthly_returns.groupby(monthly_returns.index.year).sum())
    plt.xlabel("Year")
    plt.ylabel("Returns")
    plt.title(f"{share_company} Returns Over Several Years")
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()





def compare_correlation_heatmap(company1, company2):
    # Filter the data for the selected companies
    selected_data = data[data['Symbol'].isin([company1, company2])]

    # Pivot the data for correlation analysis
    pivot_data = selected_data.pivot(index='Date', columns='Symbol', values='Close')

    # Calculate the correlation matrix
    correlation_matrix = pivot_data.corr()

    # Generate the correlation heatmap
    plt.figure(figsize=(5, 5))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title('Correlation Heatmap')
    plt.show()


    correlation = correlation_matrix.loc[company1, company2]
    if correlation > 0.7:
        # print(f"{company1} and {company2} are strongly positively correlated. It is suggested you do not invest in both stocks at once due to high risk factor")
         centered_returns = f'{company1} and {company2} are strongly positively correlated. It is suggested you do not invest in both stocks at once due to high risk factor'
         print_centered(centered_returns, 16)
    elif correlation < -0.7:
        # print(f"{company1} and {company2} are strongly negatively correlated. They tend to balanceeach other out, you can invest in either one of them or both")
        centered_returns = f'{company1} and {company2} are strongly negatively correlated. They tend to balance each other out, you can invest in either one of them or both"'
        print_centered(centered_returns, 16)
    else:
        # print(f"{company1} and {company2} are not strongly correlated. Since the risk factor is low, it is safe to invest in both stocks at once if you wish to do so")
        centered_returns = f'{company1} and {company2} are not strongly correlated. Since the risk factor is low, it is safe to invest in both stocks at once if you wish to do so'
        print_centered(centered_returns, 16)


def on_company_dropdown_change(change):
    output.clear_output()
    with output:
        plot_closing_prices(change.new)


def on_returns_button_click(button):
    output.clear_output()
    with output:
        calculate_returns_and_plot(company_dropdown.value, time_period_dropdown.value)


def on_compare_button_click(button):
    output.clear_output()
    with output:
        compare_correlation_heatmap(company_dropdown.value, compare_dropdown.value)


# Step 1: Create a dropdown widget for selecting a company
company_dropdown = widgets.Dropdown(
    options=['', *data['Symbol'].unique()],  # Include an empty option
    description='Select a company:',
       style={'description_width': 'initial'},
    layout=widgets.Layout(margin='10px auto')
)

# Step 2: Create a dropdown widget for selecting a time period
time_period_dropdown = widgets.Dropdown(
    options=['', '1d', '5d', '1month', '1year', '5years'],  # Include an empty option
    description='Select a time period:',
       style={'description_width': 'initial'},
    layout=widgets.Layout(margin='10px auto')
)


# Step 3: Create an output widget for displaying the graphs and heatmap
output = widgets.Output()


# Register the event handlers
company_dropdown.observe(on_company_dropdown_change, names='value')

# Step 4: Create a button widget for calculating returns
returns_button = widgets.Button(description="Calculate Returns", layout=widgets.Layout(margin='10px auto'))
returns_button.on_click(on_returns_button_click)

# Step 5: Create a dropdown widget for comparing with other companies
compare_dropdown = widgets.Dropdown(
     options=['', *data['Symbol'].unique()],  # Include an empty option
    description='Select another company:',
     style={'description_width': 'initial'},
    layout=widgets.Layout(margin='10px auto')
)

# Step 6: Create a button widget for comparing with another company
compare_button = widgets.Button(description="Compare Companies", layout=widgets.Layout(margin='10px auto'),    style={'description_width': 'initial'},)
compare_button.on_click(on_compare_button_click)


# Arrange the widgets and graph using HBox and VBox
dropdowns_box = VBox([company_dropdown, time_period_dropdown])
buttons_box = VBox([returns_button, compare_dropdown, compare_button])
output_box = VBox([output], layout=widgets.Layout(margin='10px auto'))  # Wrap the output in a VBox

# Set the alignment for the dropdowns and graph
dropdowns_box.layout.align_items = 'flex-start'

# Display the widgets and graph
display(VBox([dropdowns_box, buttons_box, output_box]))